In [1]:
!pip install openpyxl

You should consider upgrading via the '/Users/javierlopez/.virtualenvs/campeonatofutbol/bin/python -m pip install --upgrade pip' command.


In [2]:
import openpyxl

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
from datetime import datetime
from tqdm import tqdm
from scipy.stats import poisson, skellam
from scipy.optimize import minimize
import statsmodels.api as sm
import statsmodels.formula.api as smf
from poisson_func import * 

plt.style.use('ggplot')
plt.rcParams["figure.figsize"] = (16,9) 

## Simulación

In [4]:
new_sim = True
N_sim = 1000


df = pd.read_csv("matches_chile.csv")
df_tabla_2019 = pd.read_csv("Tabla2019.csv", index_col = "Equipo")
df_tabla_2019.sort_values(by=["PTS","DG"], ascending = False, inplace = True)
teams = df.Local.unique()
poisson_model = fit_poisson_model(df)

In [5]:
if new_sim == False:
    df_tabla_2020 = pd.read_csv("Tabla2020.csv", index_col = "Equipo")
    sim_poisson_local = pd.read_csv("sim_poisson_local.csv", index_col = [0,1])
    sim_poisson_local.columns = range(1, N_sim + 1)
    sim_poisson_visita = pd.read_csv("sim_poisson_visita.csv", index_col = [0,1])
    sim_poisson_visita.columns = range(1, N_sim + 1)
    df_posicion = pd.read_csv("df_posicion.csv", index_col = 0)
    df_desc_stats = pd.read_csv("df_desc_stats.csv", index_col = 0)
    summary_reasons = pd.read_excel("resumen_descenso.xlsx", index_col = 0)
    df_cd3 = pd.read_csv("df_cd3.csv", index_col = 0)
    df_rel_matches = pd.read_csv("df_rel_matches.csv", index_col = 0)
else:
    np.random.seed(42)
    #plot_poisson_dist(df)
    print("Construyendo tabla 2020")
    df_tabla_2020 = current_table(df, teams)
    df_posicion = pd.DataFrame(columns = ['Equipo', 'Tabla', 'n_sim', 'Posición'])
    print("Simulando partidos restantes del 2020")
    sim_poisson_local, sim_poisson_visita = poisson_tournament(df, poisson_model, N = N_sim)
    print("Summary positions")
    df_posicion = summary_positions(sim_poisson_local, sim_poisson_visita, N_sim, teams, df_tabla_2019,
                                        df_tabla_2020)
    print("Relegation stats")
    df_desc_stats = relegation_stats(N_sim, df_posicion, poisson_model)
    summary_reasons = create_summary_reasons(df_desc_stats, N_sim)
    df_cd3, df_rel_matches = cases_distribution(df_posicion, N_sim)

Construyendo tabla 2020
Simulando partidos restantes del 2020


34it [00:00, 37.84it/s]
  0%|          | 0/1000 [00:00<?, ?it/s]

0:00:01.159869
Summary positions


  0%|          | 3/1000 [00:00<00:50, 19.66it/s]

Relegation stats


  1%|▏         | 14/1000 [00:00<00:07, 138.19it/s]

10 6 197


100%|██████████| 1000/1000 [00:07<00:00, 133.72it/s]

A = X 0.384
A != X 0.616
-------------------
A = X & B = Y 0.01
A = X & B != Y 0.374
A != X & A = Y 0.061
A != X & A != Y 0.555
-------------------
A = X & B != Y & B = Z 0.002
A = X & B != Y & B != Z 0.372
A != X & A = Y & B = X 0.006
A != X & A = Y & B != X 0.055
A != X & A != Y & B = X 0.197
A != X & A != Y & B != X 0.358


## Prob de campeonar

In [6]:
df_posicion[(df_posicion["Posición"] == 1)&(df_posicion.Tabla == "Absoluta")].Equipo.value_counts()/N_sim

Universidad Católica    0.672
Unión La Calera         0.328
Name: Equipo, dtype: float64

## Prob partido definición Colo Colo vs Universidad de Chile

In [7]:
df_cc_uch = df_rel_matches[(df_rel_matches.team_1.isin(["Colo-Colo","Universidad de Chile"]))&
               (df_rel_matches.team_2.isin(["Colo-Colo","Universidad de Chile"]))]
prob_definicion_uch_cc = df_cc_uch.shape[0]/N_sim
print("Porcentaje de CC vs UCH:", 100*prob_definicion_uch_cc, "%")
df_cc_uch.head()

Porcentaje de CC vs UCH: 13.200000000000001 %


,n_sim,team_1,team_2
12,13,Colo-Colo,Universidad de Chile
43,44,Colo-Colo,Universidad de Chile
53,54,Colo-Colo,Universidad de Chile
60,61,Colo-Colo,Universidad de Chile
64,65,Colo-Colo,Universidad de Chile


## Casos de descenso de UCH

#### Prob UCH último Absoluta

In [8]:
df_ult_abs = df_posicion[(df_posicion.Tabla == "Absoluta")&
            (df_posicion["Posición"] == 18)&
            (df_posicion["Equipo"] == "Universidad de Chile")]
df_ult_abs.shape[0]/N_sim

0.0

In [9]:
df_pen_abs = df_posicion[(df_posicion.Tabla == "Absoluta")&
            (df_posicion["Posición"] == 17)&
            (df_posicion["Equipo"] == "Universidad de Chile")]
df_pen_abs.shape[0]/N_sim

0.0

In [10]:
mod_boolean = False
if mod_boolean == True:
    team = "Universidad de Chile"
    N_matches = 5
    goles_team = 0
    goles_rival = 1
    sim_poisson_local_mod, sim_poisson_visita_mod = sim_poisson_modification(team, N_matches, goles_team, 
                                                                             goles_rival, sim_poisson_local, 
                                                                             sim_poisson_visita)
    df_posicion_mod = summary_positions(sim_poisson_local_mod, sim_poisson_visita_mod, N_sim, teams, 
                                        df_tabla_2019, df_tabla_2020)
    df_desc_stats_mod = relegation_stats(N_sim, df_posicion_mod, poisson_model)
    summary_reasons_mod = summary_reasons(df_desc_stats_mod, N_sim)
    df_cd3_mod, df_rel_matches_mod = cases_distribution(df_posicion_mod, N_sim)

#### Prob UCH último Ponderada

In [11]:
df_ult_pon = df_posicion[(df_posicion.Tabla == "Ponderada")&
            (df_posicion["Posición"] == 18)&
            (df_posicion["Equipo"] == "Universidad de Chile")]
sim_ult_pon = df_ult_pon.n_sim
df_ult_pon.shape[0]/N_sim

0.0

In [12]:
df_pen_pon = df_posicion[(df_posicion.Tabla == "Ponderada")&
            (df_posicion["Posición"] == 17)&
            (df_posicion["Equipo"] == "Universidad de Chile")]
sim_pen_pon = df_pen_pon.n_sim
df_pen_pon.shape[0]/N_sim

0.011

In [13]:
summary_reasons
# ult T2020, ult TP, liguilla

,1,2,3,Prob Desc
Iquique,0.443,0.273,0.180,0.896
Universidad de Concepción,0.002,0.722,0.146,0.870
Coquimbo Unido,0.300,0.000,0.125,0.425
Colo-Colo,0.246,0.000,0.174,0.420
Universidad de Chile,0.000,0.005,0.218,0.223
Audax Italiano,0.008,0.000,0.054,0.062
Curicó Unido,0.000,0.000,0.046,0.046
La Serena,0.000,0.000,0.032,0.032
Cobresal,0.001,0.000,0.007,0.008
Everton,0.000,0.000,0.008,0.008


In [14]:
for team in teams:
    print(team)
    probs_relegation(team, df_posicion, N_sim)

Everton
O'Higgins
Antofagasta
Audax Italiano
Santiago Wanderers
Huachipato
Unión Española
Curicó Unido
Colo-Colo
Unión La Calera
Universidad de Concepción
Palestino
Universidad de Chile
Cobresal
Universidad Católica
Iquique
La Serena
Coquimbo Unido


<Figure size 1152x648 with 0 Axes>